# Proszę dużą dozę sceptyzmu przy przyswajaniu informacji z tego notebooka! Nie mam w temacie doświadczenia, tylko lektura (wiki i książka) i przykład.
# Jeśli gdzieś się pomyliłem - proszę o wyrozumiałość.

# minimum teorii na temat tego czym jest PCA

 
 
### PCA - Principal Component Analysis (Analiza Głównych Składowych)

* https://en.wikipedia.org/wiki/Principal_component_analysis
* https://pl.wikipedia.org/wiki/Analiza_g%C5%82%C3%B3wnych_sk%C5%82adowych

### w dużym skrócie:
* PCA przydaje się, jako narzędzie pozwalające zredukować wielkość (ilość wymiarów) data-setu przy zachowaniu możliwie wysokiej ilości informacji, co może np. pozwolić oszczędzić czas potrzebny na wykonanie uczenia modelu
* WAŻNE - przy stosowaniu PCA należy - analogicznie do KNN - znormalizować poszczególne zmienne !
* po przepuszczeniu data-setu przez algorytm PCA otrzymamy mniejszy data-set (macierz), ale występujące w nim zmienne nie odpowiadają wprost zmiennyn wejściowym



* w naszym projekcie z uwagi na małą ilość zmiennych i niskie korelacje zastosowanie PCA wydaje się nie mieć sensu

## Przykład w oparciu o breast cancer dataset i klasyfikację KNN

In [1]:
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA

In [2]:
#wczytanie i sprawdzenie "co jest w datasecie"

bc = load_breast_cancer()
print(bc.keys())
print(bc.data.shape)
print(bc.data[0].size)
print(bc.target.shape)
print(bc.feature_names)

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])
(569, 30)
30
(569,)
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [3]:
print(bc.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [4]:
x_df = pd.DataFrame(bc['data'], columns = bc['feature_names'])
y = pd.Series(bc['target'])

In [5]:
x_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [6]:
y

0      0
1      0
2      0
3      0
4      0
      ..
564    0
565    0
566    0
567    0
568    1
Length: 569, dtype: int64

In [7]:
#podział na dane testowe i treningowe

X_train, X_test,y_train,y_test = train_test_split(bc.data[:], bc.target, test_size = 0.25, random_state = 42)
print(X_train.shape)
print(X_test.shape)

(426, 30)
(143, 30)


In [8]:
# normalizacja danych za pomocą MinMax scalera

# "skalibrowanie scalera na dane treningowe"
scaler = MinMaxScaler()
scaler.fit(X_train)  


#normalizacja danych testowcyh i traningowych
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

#wyświetlenie i kontrola wielkości macierzy
print('X_train \n',X_train.shape,'\n',X_train,'\n')
print('X_train_norm \n',X_train_norm.shape,'\n',X_train_norm,'\n')
print('X_test \n',X_test.shape,'\n',X_test, '\n')
print('X_test_norm \n',X_test_norm.shape,'\n',X_test_norm)

X_train 
 (426, 30) 
 [[1.289e+01 1.312e+01 8.189e+01 ... 5.366e-02 2.309e-01 6.915e-02]
 [1.340e+01 2.052e+01 8.864e+01 ... 2.051e-01 3.585e-01 1.109e-01]
 [1.296e+01 1.829e+01 8.418e+01 ... 6.608e-02 3.207e-01 7.247e-02]
 ...
 [1.429e+01 1.682e+01 9.030e+01 ... 3.333e-02 2.458e-01 6.120e-02]
 [1.398e+01 1.962e+01 9.112e+01 ... 1.827e-01 3.179e-01 1.055e-01]
 [1.218e+01 2.052e+01 7.722e+01 ... 7.431e-02 2.694e-01 6.878e-02]] 

X_train_norm 
 (426, 30) 
 [[0.2546158  0.11531958 0.24164177 ... 0.18439863 0.14665878 0.11961682]
 [0.27959254 0.36557322 0.28965713 ... 0.704811   0.39818648 0.47355036]
 [0.25804398 0.29015894 0.25793143 ... 0.22707904 0.32367435 0.14776195]
 ...
 [0.32317939 0.2404464  0.30146536 ... 0.11453608 0.17602996 0.05222109]
 [0.30799745 0.33513696 0.30729834 ... 0.62783505 0.31815494 0.42777213]
 [0.21984426 0.36557322 0.20842225 ... 0.25536082 0.22255076 0.11648016]] 

X_test 
 (143, 30) 
 [[1.247e+01 1.860e+01 8.109e+01 ... 1.015e-01 3.014e-01 8.750e-02]
 [1.894

In [9]:
# klasyfikacja za pomocą KNN i ocena skuteczności klasyfiakcji

classifier = KNeighborsClassifier(n_neighbors=4)
classifier.fit(X_train_norm, y_train)

preds = classifier.predict(X_test_norm)

sc = accuracy_score(y_test, preds)
print('skuteczność na danych testowych:', sc)
print(confusion_matrix(y_test,preds))
print(classification_report(y_test,preds))

skuteczność na danych testowych: 0.965034965034965
[[52  2]
 [ 3 86]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.95        54
           1       0.98      0.97      0.97        89

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.97      0.97      0.97       143



## Najpierw spróbuję zmniejszyć ilość zmiennych za pomocą PCA, w dalszej kolejności przeprowadzić klasyfikację takim samym modelem KNN, ale bazując na mniejszej liczbie zmiennych i porównać uzyskaną skuteczność

In [10]:
pca = PCA(n_components=0.9)

# parametr n_components oznacza oznacza "ile wymiarów PCA ma uzyskać"
# jeżeli wybierzemy inta - do tylu wymiarów sprowadzi naszą X-wymiarową macierz (nie wiemy "ile informacji" zostanie zachowanej)
# jeżeli wpiszemy floata od 0 do 1, to nie wiemy jaką redukację otrzymamy (ile dostaniemy wymiarów), możemy określić "jak dużo informacji należy zachować" 
# mam świadomość, że powyższe tłumaczenie jest nieprecyzyjne i "na chłopski rozum", ale dawno nie miałem styczności z algebrą liniową i nie chcę nic pokręcić
# po więcej info odsyłam do wikipedii - link na górze, można też doczytać w książce, którą udostępniłem

# co do zasady - jeśli powyżej wybraliśmy wartość 0.9, to oznacza, że dla wyliczonych wartości własnych macierzy kowariancji, bierzemy tyle największych,
# żeby suma wziętych była większa równa 0.9 całości wyznaczonych (algorytm wyznacza 1 wartość własną dla każdej z bazowych zmiennych)

# za wiki: Im wyższa wartość własna tym odpowiadający jej wektor własny jest słabiej skorelowany z pozostałymi.

# z tego wynika, że algorytm identyfikuje "najbardziej charakterystyczne" zmienne i pozwala odrzucić te, które są
# bardziej skorelowane, jako "powtarzające te same informacje"


# OPERUJEMY NA ZNORMALIZOWANYCH DANYCH

X_train_pca = pca.fit_transform(X_train_norm)
X_test_pca = pca.transform(X_test_norm)

#kontrola wielkości macierzy i porównanie z niezrudukawanym datasetem
print('X_train_pca \n',X_train_pca.shape,'\n',X_train_pca)

print('X_train_norm \n',X_train_norm.shape,'\n',X_train_norm,'\n')


print('X_test_pca \n',X_test_pca.shape,'\n')



X_train_pca 
 (426, 6) 
 [[-0.64239154 -0.2869642  -0.07838583 -0.2588534   0.24531343  0.07208835]
 [ 0.46413174  0.38525073 -0.27283272  0.05657963 -0.07465243  0.01337816]
 [-0.34113621 -0.06345178  0.1458204   0.04561338  0.15067456  0.29640418]
 ...
 [-0.67370933 -0.47880723 -0.03871123 -0.07490048  0.16248726  0.14044478]
 [ 0.25289968  0.14892849 -0.38280419  0.09611908 -0.02018387 -0.09580602]
 [-0.51111414 -0.1951197  -0.0556636   0.26971743 -0.01296577  0.02777509]]
X_train_norm 
 (426, 30) 
 [[0.2546158  0.11531958 0.24164177 ... 0.18439863 0.14665878 0.11961682]
 [0.27959254 0.36557322 0.28965713 ... 0.704811   0.39818648 0.47355036]
 [0.25804398 0.29015894 0.25793143 ... 0.22707904 0.32367435 0.14776195]
 ...
 [0.32317939 0.2404464  0.30146536 ... 0.11453608 0.17602996 0.05222109]
 [0.30799745 0.33513696 0.30729834 ... 0.62783505 0.31815494 0.42777213]
 [0.21984426 0.36557322 0.20842225 ... 0.25536082 0.22255076 0.11648016]] 

X_test_pca 
 (143, 6) 



In [11]:
# klasyfikacja za pomocą KNN i ocena skuteczności klasyfiakcji

classifier_pca = KNeighborsClassifier(n_neighbors=4)
classifier_pca.fit(X_train_pca, y_train)

preds_pca = classifier_pca.predict(X_test_pca)

sc_pca = accuracy_score(y_test, preds_pca)

print('skuteczność bez PCA na danych testowych:', sc)

print('skuteczność po PCA na danych testowych:', sc_pca)
print(confusion_matrix(y_test,preds_pca))
print(classification_report(y_test,preds_pca))

skuteczność bez PCA na danych testowych: 0.965034965034965
skuteczność po PCA na danych testowych: 0.965034965034965
[[52  2]
 [ 3 86]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.95        54
           1       0.98      0.97      0.97        89

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted avg       0.97      0.97      0.97       143



## z powyższego eksperymentu wynika, że dla testowanego datasetu można zredukować ilość zmiennych, na których uczy się model do 6 zamiast 30 i klasyfikacja wciąż będzie równie skuteczna
## Oczywiście, żeby model zadziałał dla przychodzących danych należy pełne 30 danych przepuścić przez normalizację i pca. 
### To też nie jest tak, że któreś zmienne pomijamy, bo wartości otrzymanych po przepuszczeniu przez PCA 6 zależą od wszystkich 30, które są wejściem